<a href="https://colab.research.google.com/github/sofpast/bert_sentiment_analysis/blob/main/Text_Classification_by_Simple_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification By Simple LSTM

In [ ]:
!pip install torchtext
!pip install torch

In [1]:
import spacy
import torchtext
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch

## Loading & Data Cleaning

In [27]:
if torch.cuda.is_available():
  device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
  device = 'cpu'

print(f"using device: {device}")


using device: cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ln -s /content/drive/MyDrive/DeepLearningandNLP

In [6]:
cd DeepLearningandNLP

/content/drive/MyDrive/DeepLearningandNLP


In [ ]:
tweetsDF.head(3)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [ ]:
tweetsDF.groupby(tweetsDF[0]).count()
# it seems only binary values (0 and 4)

,1,2,3,4,5
0,,,,,
0,800000,800000,800000,800000,800000
4,800000,800000,800000,800000,800000


In [ ]:
tweetsDF[0].value_counts()

0    800000
4    800000
Name: 0, dtype: int64

In [ ]:
tweetsDF['sentiment_cat'] = tweetsDF[0].astype('category')
tweetsDF.head(3)

,0,1,2,3,4,5,sentiment_cat
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0


In [ ]:
tweetsDF['sentiment_cat'].value_counts()

0    800000
4    800000
Name: sentiment_cat, dtype: int64

In [ ]:
tweetsDF.info()

In [ ]:
tweetsDF['sentiment'] = tweetsDF['sentiment_cat'].cat.codes
tweetsDF['sentiment'].value_counts()

0    800000
1    800000
Name: sentiment, dtype: int64

In [ ]:
# tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')
# tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes
tweetsDF.to_csv("train-processed.csv", header=None, index=None)
tweetsDF.sample(10000).to_csv("train-processed-sample.csv", header=None, index=None)


In [3]:
from torch.utils.data import random_split, Dataset

In [4]:
import pandas as pd
import torch

## Create our Dataset and DataLoaders

In [ ]:
# !pip install torchtext==0.6.0

In [9]:
from torchtext.data import get_tokenizer
#only work for torchtext==0.6.0
TWEET = torchtext.data.Field(tokenize=get_tokenizer('spacy', language='en_core_web_sm'), lower=True)

In [10]:
torchtext.data.Field()

In [13]:
import pandas as pd

df = pd.read_csv("/content/DeepLearningandNLP/train_processed_2.csv")
df.head(3)

,Unnamed: 0,score,id,date,query,name,tweet,category,label
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,0
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0,0
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,0


In [20]:
df.columns

Index(['score', 'id', 'date', 'query', 'name', 'tweet', 'category', 'label'], dtype='object')

In [21]:
# df = df.drop("Unnamed: 0", axis=1)
df.head(1000).to_csv("training_3.csv", index=False)

In [6]:
from torchtext.data import TabularDataset

In [22]:
import torch
import torchtext
from torchtext.data.utils import get_tokenizer


LABEL = torchtext.data.LabelField()
# TWEET = torchtext.fields.Field('spacy', tokenizer_language='en_core_web_sm', lower=True)
TWEET = torchtext.data.Field(tokenize=get_tokenizer('spacy', language='en_core_web_sm'), lower=True)

fields = [('score',None), ('id',None), ('date',None), ('query',None),
          ('name',None), ('tweet', TWEET), ('category',None), ('label',LABEL)]

twitterDataset = torchtext.data.TabularDataset(
    path="/content/DeepLearningandNLP/training_3.csv",
    format="CSV",
    fields=fields,
    skip_header=False)


In [23]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label')

(len(train),len(test),len(valid))

(601, 200, 200)

In [25]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
TWEET.vocab.freqs.most_common(10)

[('i', 470),
 ('.', 391),
 ('!', 269),
 (' ', 233),
 ('to', 227),
 ('the', 200),
 (',', 193),
 ('my', 169),
 ('a', 143),
 ('and', 130)]

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

In [27]:
train_iterator, valid_iterator, test_iterator

(<torchtext.data.iterator.BucketIterator at 0x7f0b77a2fa30>,
 <torchtext.data.iterator.BucketIterator at 0x7f0b77a2f0d0>)

## Our First LSTM

In [67]:
import torch.nn as nn

class OurFirstLSTM(nn.Module):
  def __init__(self, hidden_size, embedding_dim, vocab_size):
    super(OurFirstLSTM, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim) # n-dimensional vector for embedding
    self.encoder = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_size, num_layers=1) # LSTM layer with hidden-size is the dimension of hidden state, num_layers = no of LSTM layer in the stack
    self.predictor = nn.Linear(hidden_size, 2) # fully connected layer with 2 outputs (possible classes: negative, positive or neutral)

  def forward(self, seq):
    output, (hidden,_) = self.encoder(self.embedding(seq))
    preds = self.predictor(hidden.squeeze(0))
    return preds

In [53]:
device = 'cpu'

In [68]:
model = OurFirstLSTM(10, 300, 20002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 10, num_layers=10)
  (predictor): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
# class OurFirstLSTM(nn.Module):
#     def __init__(self, hidden_size, embedding_dim, vocab_size):
#         super(OurFirstLSTM, self).__init__()

#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.encoder = nn.LSTM(input_size=embedding_dim,
#                 hidden_size=hidden_size, num_layers=1)
#         self.predictor = nn.Linear(hidden_size, 2)

#     def forward(self, seq):
#         output, (hidden,_) = self.encoder(self.embedding(seq))
#         preds = self.predictor(hidden.squeeze(0))
#         return preds

# model = OurFirstLSTM(100,300, 20002)
# model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

In [60]:
device

'cpu'

## Training

In [65]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):

        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)


        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)

        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [66]:
train(10, model, optimizer, criterion, train_iterator, valid_iterator)

Epoch: 1, Training Loss: 4.01, Validation Loss: 0.60
Epoch: 2, Training Loss: 0.58, Validation Loss: 0.23
Epoch: 3, Training Loss: 0.42, Validation Loss: 0.15
Epoch: 4, Training Loss: 0.44, Validation Loss: 0.12
Epoch: 5, Training Loss: 0.44, Validation Loss: 0.11
Epoch: 6, Training Loss: 0.42, Validation Loss: 0.09
Epoch: 7, Training Loss: 0.41, Validation Loss: 0.08
Epoch: 8, Training Loss: 0.38, Validation Loss: 0.08
Epoch: 9, Training Loss: 0.42, Validation Loss: 0.08
Epoch: 10, Training Loss: 0.43, Validation Loss: 0.07


## Making predictions

In [33]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    model.eval()
    return categories[model(processed).argmax().item()]

In [36]:
tweet = "happy!"
classify_tweet(tweet)

'Negative'

## Data Augmentation

In [42]:
import random
def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining

In [43]:
words = "today is a happy day"
random_deletion(words)

['t', 'o', 'a', ' ', 'i', 's', ' ', ' ', 'a', 'p', 'd', 'a']

In [49]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence = list(sentence)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return "".join(sentence)

In [50]:
sentence = "today is a happy day"
random_swap(sentence)

'aaday is t h ppydaoy'

In [ ]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

In [ ]:
# Install googletrans version 3.1.0a0 (temporary fix for #57)
!pip install googletrans==3.1.0a0

In [ ]:
import googletrans
import random

translator = googletrans.Translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, dest='fr')
fr_text = [t.text for t in translations_fr]
translations_en = translator.translate(fr_text, dest='en')
en_text = [t.text for t in translations_en]
print(en_text)

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)